# Saturable Absorption Demo
In this notebook I will try to simulate the property called saturable absorption that is present in some materials like dyes and semiconductors. 

## What is Saturable Absorption?
Essentially, this is the property where the rate of absorption slows as the incident light becomes more intense. In other words, as the light entering the material becomes brighter, the amount of light that makes it through to the other side of the material per unit of light becomes less. So in short, you get diminishing returns.

## Importance
So why do we care about this property? Well, it turns out that this property introduces a nonlinearity - something that is vital to the operation of Multi-Layer Perceptron neural networks. This enables us to implement an activation function using light. Neat, right?

## Applications
Recently there has been some interest in photonic computing - basically computers that use light instead of electrons to do their computations. The benefits of photonic computing are appealing, it offers significant speedup and lowered power consumption over conventional computers. However, current applications of photonic seems to be rather limited, and the applications that I have seen so far have been very specific. 

One application that is of much interest to those who dabble in deep learning is the optical neural network (ONN). An ONN has been implemented using a nanophotonic prossesor by a group of researchers at MIT. The chip that they used is made up of a few key components: waveguides, phase shifters, and saturable absorbers. The first two are used to perform matrix multiplication and the saturable

## The Effect of Saturable Absorption
The effect of saturable absoption can be expressed as:
$$\sigma\tau_sI_0=\frac{1}{2}\frac{ln(\frac{T_m}{T_0})}{1-T_m}$$
where $\sigma$ is the absorption cross section per molecule, $\tau_s$ is the lifetime of molecules in the excited state, $I_0$ is the peak incident intensity, $T_m$ is the maximum transmittance, and $T_0$ is the initial transmittance.

## Some Values
In order to simulate saturable absorption, I need two things: the mathematical model that describes it, and some values to plug into said model. If you have read this, then you will have noticed that we already have the model, so now it is just a matter of finding some suitable values.

The supplementary for the paper titled 'Deep learning with coherent nanophotonic circuits' provides us with some ballpark figures for the $\sigma$ and $\tau_s$ terms of the equation. Those values are:
$$\sigma\approx10^{-15} \:\text{(cm)}^2$$
$$\tau_s=10^{-9} \:\text{(sec)}$$

Some digging into the paper titled 'Monolayer Graphene as a Saturable Absorber in a Mode-Locked Laser' provides us with a few more values from their experiments (these are the best estimates I could manage with my current understanding):
$$T_0=-35.1 \:\text{dBm}$$
$$T_m=-2.64 \:\text{dBm}$$
Note that $T_0$ is roughly equivalent to 309 nW and $T_m$ is roughly equivalent to 0.55 mW.

These values were achieved using a femtosecond Ti:sapphire laser system with an incident intensity in the closed interval $[-30 \:\text{dBm}, -2.5 \:\text{dBm}]$.

With all these values, I think I have enough to reproduce the graph in the supplementary of 'Deep learning with coherent nanophotonic circuits' (Figure S3).